In [17]:
import torch
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

IMPORTING MNIST DATASET

In [2]:
from sklearn.datasets import fetch_openml

# Load MNIST dataset from openml
mnist = fetch_openml('mnist_784', version=1)

# Get data and labels
X, y = mnist['data'], mnist['target']

# Split into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Display the shapes
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (52500, 784)
y_train shape: (52500,)
x_test shape: (17500, 784)
y_test shape: (17500,)


In [15]:
X = torch.tensor(x_train.values, dtype=torch.float32)
x = torch.tensor(x_test.values, dtype=torch.float32)
Y = torch.tensor(y_train.values.astype(int), dtype=torch.long)
y = torch.tensor(y_test.values.astype(int), dtype=torch.long)

In [23]:
# Create DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
train_dataset = TensorDataset(X,Y)
test_dataset = TensorDataset(x, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x




model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")


correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')


Epoch 1/10, Loss: 0.3090018945065651
Epoch 2/10, Loss: 0.12324735125479112
Epoch 3/10, Loss: 0.08502726787916912
Epoch 4/10, Loss: 0.06411333394927551
Epoch 5/10, Loss: 0.048055640879062465
Epoch 6/10, Loss: 0.03815509838340898
Epoch 7/10, Loss: 0.03318602206109517
Epoch 8/10, Loss: 0.026584571887166893
Epoch 9/10, Loss: 0.02245182095782701
Epoch 10/10, Loss: 0.020892977935118472
Accuracy on test set: 97.54857142857144%
